In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
# from yellowbrick.regressor import ResidualsPlot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
cols2=[] # 繪圖用欄位名稱
for i in data1.columns:
    cols2.append(i)
cols2.pop()

## 訓練

In [ ]:
data1 = pd.read_csv('./exchange_ratePrice10000000Lot1000000Sqft10000_ptta.csv')
data1 = data1.drop(columns = ['_25k', '25k_60k','60k_150k', '150k_',
                              'male', 'female', 'sexRatio',
                              'White', 'African', 'Asian', 
                              'Hawaiian', 'Indian', 'Others',
                              'WhiteRatio','MaleMedianAge',
                              'FemaleMedianAge'])
data1['zipcode'] = data1['zipcode'].astype(str)
data1['zipcode'] = data1['zipcode'].astype('category')
X = data1.iloc[:,:-1].values
y= data1.iloc[:,-1].values
# X2 = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test =  train_test_split(X, y,
                     test_size=0.20)
xgb = XGBRegressor(
#     objective='reg:squarederror',
    learning_rate=0.2,
    n_estimators=275, 
    max_depth=6,
    min_child_weight=1,
    colsample_bytree=0.95,
    subsample=0.8,
    reg_alpha=4,
    reg_lambda=3,

) 
xgb.fit(X_train,y_train)
print(time.process_time() - start)

y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)
print('測試正確率: %.3f' % xgb.score(X_test, y_test))
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

In [ ]:
# visualizer繪製殘插圖
visualizer = ResidualsPlot(pipe_lr)
visualizer.fit(X_train,y_train)
visualizer.score(X_test, y_test)
visualizer.show()

In [ ]:
# 第二種殘插圖看喜歡哪個
plt.scatter(y_train_pred,  y_train_pred - y_train,
            c='steelblue', marker='o', edgecolor='white',
            label='Training data')
plt.scatter(y_test_pred,  y_test_pred - y_test,
            c='limegreen', marker='s', edgecolor='white',
            label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=50, color='black', lw=2)
plt.xlim([100000, 10000000])
plt.tight_layout()

# plt.savefig('demo6_1.png', dpi=300)
plt.show()

In [ ]:
# 重要特徵長條圖
importances = []
for i in xgb.feature_importances_:
    importances.append(i)
# print(importances)
# print(cols2)
feature_dict = {"feature": cols2,"importans":importances}
feature_importance_df =pd.DataFrame(feature_dict)
# feature_importance_df
plt.figure(figsize=(12, 6))
sns.barplot(x="importans", y="feature", data=feature_importance_df)
plt.title('RF features importance:')

In [ ]:
# 兩種圖看喜歡哪一個
plt.figure(num=None, figsize=(20, 22), dpi=80, facecolor='w', edgecolor='k')

feat_importances = pd.Series(xgb.feature_importances_, index= cols2)
feat_importances.nlargest(56).plot(kind='barh')

## 網格搜索(找最佳參數，會跑很久)

In [ ]:
data1 = pd.read_csv('./testData.csv')
data1 = data1.drop(columns = ['_25k', '25k_60k','60k_150k', '150k_',
                              'male', 'female', 'sexRatio',
                              'White', 'African', 'Asian', 'Hawaiian', 'Indian', 'Others',
                              'WhiteRatio','MaleMedianAge',
                              'FemaleMedianAge'])
X = data1.iloc[:,:-1].values
y= data1.iloc[:,-1].values
X2 = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test =  train_test_split(X2, y,
                     test_size=0.20)

xgb = XGBRegressor(
#     objective='reg:squarederror',
    learning_rate=0.2,
    n_estimators=275, 
    max_depth=6,
    min_child_weight=1,
    colsample_bytree=0.95,
    subsample=0.8,
    reg_alpha=4,
    reg_lambda=3,

) 

cv_params = {'n_estimators': [80,85,100,120]}
xgb_cv = GridSearchCV(xgb,param_grid=cv_params, scoring='r2', cv=3, verbose=2, n_jobs=-1)
xgb_cv.fit(X_train,y_train)



# print result
y_train_pred = xgb_cv.predict(X_train)
y_test_pred = xgb_cv.predict(X_test)
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))
# for i in grid_scores_:
#   print(i)
print(xgb_cv.best_score_)
print(xgb_cv.best_params_)

print("Best score: %0.3f" % xgb_cv.best_score_) # 模型最佳績效
print("Best parameters set:") # 窮舉後的最佳參數
best_parameters = xgb_cv.best_estimator_.get_params()
for param_name in sorted(cv_params.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

## k折驗證

In [ ]:
data1 = pd.read_csv('./exchange_ratePrice10000000Lot1000000Sqft10000_ptta.csv')
data1 = data1.drop(columns = ['_25k', '25k_60k','60k_150k', '150k_',
                              'male', 'female', 'sexRatio',
                              'White', 'African', 'Asian', 
                              'Hawaiian', 'Indian', 'Others',
                              'WhiteRatio','MaleMedianAge',
                              'FemaleMedianAge'])
data1['zipcode'] = data1['zipcode'].astype(str)
data1['zipcode'] = data1['zipcode'].astype('category')
X = data1.iloc[:,:-1].values
y= data1.iloc[:,-1].values
X_train, X_test, y_train, y_test =  train_test_split(X, y,
                     test_size=0.20)
xgb = XGBRegressor(
#     objective='reg:squarederror',
    learning_rate=0.2,
    n_estimators=275, 
    max_depth=6,
    min_child_weight=1,
    colsample_bytree=0.95,
    subsample=0.8,
    reg_alpha=4,
    reg_lambda=3,

) 
pipe_lr = make_pipeline(StandardScaler(),
                        xgb)

# pipe_lr.fit(X_train, y_train)
# y_pred = pipe_lr.predict(X_test)
# print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

kfold = StratifiedKFold(n_splits=6).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Acc: %.3f' % (k+1,score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
import matplotlib.pyplot as plt
k_range = range(1,7)

plt.plot(k_range,scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()